In [1]:
from mypydrive.auth import GoogleAuth
from mypydrive.drive import GoogleDrive
# from extGoogleDrive import extGoogleDrive as GoogleDrive
from pprint import pprint
import pyAesCrypt
import io
import os

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1041234379035-ua0eveg1b0rb811hfmnj9n0m348a632h.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8090%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


https://console.cloud.google.com/apis/credentials?authuser=1&folder=&organizationId=&project=quickstart-1579349870305

In [11]:
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()
for file in fileList:
    print('Title: %s, ID: %s, mime: %s' % (file['title'], file['id'], file['mimeType']))
    # Get the folder ID that you want
    if(file['title'] == "To Share"):
        fileID = file['id']

Title: test, ID: 1B3RlEbIGqnV3QrU856Cn4e1pNtEeuPNb, mime: application/vnd.google-apps.folder
Title: Colab Notebooks, ID: 1P-5ivq0k-GOdS3waqSF42eWjkKn4p2ka, mime: application/vnd.google-apps.folder
Title: py_backup, ID: 1YJkqN-qtRHXnL4GvT3icGyl05_dcYTE6, mime: application/vnd.google-apps.folder
Title: other, ID: 15UZpLHZ-koagoj_T7RDYStcpKNXFcC5h, mime: application/vnd.google-apps.folder
Title: Backup, ID: 1prgjfxfDBJOL_FxhDekqEk7Z1lXoEGGO, mime: application/vnd.google-apps.folder
Title: orbi, ID: 1Qxre8vhr4qIsP2Sw2Qo8SM8SAzU9XQjm, mime: application/vnd.google-apps.folder
Title: Share, ID: 1d5WSWmw8yJ-q4YwGq8MgeuMUM50NFXZP, mime: application/vnd.google-apps.folder


In [12]:
def search_droot(folder):
    fileList = drive.ListFile({'q': "'root' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()
    return [{'title': f['title'], 'id': f['id']} for f in fileList if folder in f['title']]

search_droot('')

[{'title': 'test', 'id': '1B3RlEbIGqnV3QrU856Cn4e1pNtEeuPNb'},
 {'title': 'Colab Notebooks', 'id': '1P-5ivq0k-GOdS3waqSF42eWjkKn4p2ka'},
 {'title': 'py_backup', 'id': '1YJkqN-qtRHXnL4GvT3icGyl05_dcYTE6'},
 {'title': 'other', 'id': '15UZpLHZ-koagoj_T7RDYStcpKNXFcC5h'},
 {'title': 'Backup', 'id': '1prgjfxfDBJOL_FxhDekqEk7Z1lXoEGGO'},
 {'title': 'orbi', 'id': '1Qxre8vhr4qIsP2Sw2Qo8SM8SAzU9XQjm'},
 {'title': 'Share', 'id': '1d5WSWmw8yJ-q4YwGq8MgeuMUM50NFXZP'}]

In [ ]:
# Create GoogleDriveFile instance with title 'Hello.txt'.
file1 = drive.CreateFile()
# file1.SetContentFile('test.jpg')
file1.SetContentString(b'qweradf')
file1.Upload() # Upload the file.

In [ ]:
# file1.SetContentString(img.dencode())
file1 = drive.CreateFile()
file1.SetContentBinary(open("test.jpg", "rb"), "ani.jpg")
file1.Upload() # Upload the file.

In [ ]:
def SetContentBinary(self, data, filename):
    """Set content of this file from a file.
    Opens the file specified by this method.
    Will be read, uploaded, and closed by Upload() method.
    Sets metadata 'title' and 'mimeType' automatically if not specified.
    :param filename: name of the file to be uploaded.
    :type filename: str.
    """
    self.content = data
    if self.get('title') is None:
        self['title'] = filename
    if self.get('mimeType') is None:
        self['mimeType'] = mimetypes.guess_type(filename)[0]

def GetContentBinary(self, filename, mimetype=None, remove_bom=False):
    """Save content of this file as a local file Dimon.

    :param filename: name of the file to write to.
    :type filename: str
    :param mimetype: mimeType of the file.
    :type mimetype: str
    :param remove_bom: Whether to remove the byte order marking.
    :type remove_bom: bool
    :raises: ApiRequestError, FileNotUploadedError, FileNotDownloadableError
    """
    if self.content is None or \
                    type(self.content) is not io.BytesIO or \
                    self.has_bom == remove_bom:
        self.FetchContent(mimetype, remove_bom)
    return self.content.getvalue()

### Test encription IO

In [ ]:
import pyAesCrypt
import io

bufferSize = 64 * 1024
password = "foopassword"
# binary data to be encrypted
pbdata = b"This is binary plaintext"
# input plaintext binary stream
fIn = io.BytesIO(pbdata)
# initialize ciphertext binary stream
fCiph = io.BytesIO()
# initialize decrypted binary stream
fDec = io.BytesIO()
# encrypt stream
pyAesCrypt.encryptStream(fIn, fCiph, password, bufferSize)
# print encrypted data
print("This is the ciphertext:\n" + str(fCiph.getvalue()) +  f" {len(str(fCiph.getvalue()))}")

# get ciphertext length
ctlen = len(fCiph.getvalue())

# go back to the start of the ciphertext stream
fCiph.seek(0)

# decrypt stream
pyAesCrypt.decryptStream(fCiph, fDec, password, bufferSize, ctlen)

# print decrypted data
print("Decrypted data:\n" + str(fDec.getvalue()))

In [ ]:
file1 = drive.CreateFile()
file1.SetContentBinary(fCiph, "ctest.ciph")
file1.Upload() # Upload the file.

In [ ]:
# decrypt stream
fCiphFile = open("ctest.ciph", "rb")
pyAesCrypt.decryptStream(fCiphFile, fDec, password, bufferSize, ctlen)

# print decrypted data
print("Decrypted data:\n" + str(fDec.getvalue()))

In [ ]:
def decript_and_save(fname, bufferSize = 64 * 1024, password = "foopassword"):
    pyAesCrypt.decryptFile(fname, fname.replace(".ciph", ""), password, bufferSize)



"1M5j2NCzg-o-iq2r6JakvEeo0Fwh6GxG-"
decript_stream_and_save('asd.txt', "1M5j2NCzg-o-iq2r6JakvEeo0Fwh6GxG-")

# dfile = drive.CreateFile({'id': "1M5j2NCzg-o-iq2r6JakvEeo0Fwh6GxG-"})
# fCiph = dfile.GetContentBinary("asd")
# print(fCiph)
# decript_and_save("test2.jpg.ciph")

### Useful functions

In [3]:
def driveListFolder(parent):
    filelist=[]
    file_list = drive.ListFile({'q': f"'{parent}' in parents and trashed=false"}).GetList()
    for f in file_list:
        if f['mimeType']=='application/vnd.google-apps.folder': # if folder
            filelist.append({"id": f['id'],
                             "isfolder": True,
                             "title": f['title'],
                             "list": driveListFolder(f['id'])
                            })
        else:
            filelist.append({"id": f["id"],
                             "title": f['title'], 
                             "isfolder": False
                            })
    return filelist

def localListFolder(parent, level=0, verbose=False,
                   ignore=['.git', ".ipynb_checkpoints", "__pycache__"]):
    cwd = os.getcwd()  # restore cwd after recursion is finished
    os.chdir(parent) 
    indent = '    '
    folder_dict={"isfolder": True, 
                 "title": parent,
                 "children": []}
    file_list = [asd for asd in os.listdir(".") if asd not in ignore]
    for f in file_list:
        if os.path.isdir(f): # if folder
            if verbose: print(f"{indent*level}{parent}: folder {f}")
            folder_dict['children'].append(localListFolder(f, level=level+1, verbose=verbose))
        else:
            if verbose: print(f"{indent*level}{parent}: file   {f}")
            folder_dict['children'].append({"isfolder": False, 
                                            "title": f})

    os.chdir(cwd)
    return folder_dict

# pprint(localListFolder('.'))
# dlist = driveListFolder(asd.root_id)
# pprint(dlist)
# os.getcwd()

In [4]:
# dlist = driveListFolder(asd.root_id)

def driveList2localDict(dlist, id, parent, level=0, verbose=True, indent = "|---"):
    folder_dict={"isfolder": True, 
                 "id": id,
                 "title": parent,
                 "children": []}
    for f in dlist:
        if f["isfolder"]:
            if verbose: print(f"{indent*level}{parent}: FOLDER {f['title']}")
            folder_dict['children'].append(driveList2localDict(f['list'], f['id'], f['title'], level+1, verbose, indent))
        else:
            if verbose: print(f"{indent*level}{parent}: FILE   {f['title']}")
            folder_dict['children'].append({"isfolder": False, 
                                            "id": f["id"],
                                            "title": f['title']})
    return folder_dict
            
# pprint(driveList2localDict(dlist, "Asd", "py_backup", verbose=False))

In [5]:
def encript_and_upload(fname, parent_id, bufferSize = 64 * 1024, password = "foopassword"):
    fIn = open(fname, "rb")
    fCiph = io.BytesIO()
    pyAesCrypt.encryptStream(fIn, fCiph, password, bufferSize)
    
    file = drive.CreateFile({'title': fname, 
                             'parents':[{'id': parent_id}]
                             })
    file.SetContentBinary(fCiph)
    file.Upload() # Upload the file.
    
    fIn.close()
    return file['id']

def decript_stream_and_save(drive_file_id, bufferSize = 64 * 1024, password = "foopassword"):
    fDec = io.BytesIO()
    dfile = drive.CreateFile({'id': drive_file_id})
    fCiph = io.BytesIO(dfile.GetContentBinary())
    pyAesCrypt.decryptStream(fCiph, fDec, password, bufferSize, len(fCiph.getvalue()))
    with open(dfile['title'], 'wb') as f:
        f.write(fDec.getvalue())

In [6]:
def mkdir(folder_name):
    try: 
        os.mkdir(folder_name)
    except FileExistsError:
        pass

### Drive folders

In [7]:
class DriveHandler:
    
    def __init__(self, password = "foopassword", bufferSize = 64 * 1024):
        self.root_id = "1YJkqN-qtRHXnL4GvT3icGyl05_dcYTE6"
#         self.root = drive.CreateFile({'id': self.root_id})
        self.cache = None
        self.bufferSize = bufferSize
        self.password = password
    
    def create_folder(self, child_name=None, parent_id=None):
        if parent_id is None:
            parent_id = self.root_id
        if child_name is None:
            child_name = "New Folder"
        child_folder = drive.CreateFile({'title': child_name, 
                                         'parents':[{'id':parent_id}], 
                                         'mimeType' : 'application/vnd.google-apps.folder'})
        child_folder.Upload()
        return child_folder['id']
    
    def create_encrypted_file(self, fname, parent_id):
        if parent_id is None:
            parent_id = self.root_id
        return encript_and_upload(fname, parent_id, self.bufferSize, self.password)
    
    def upload_folder_tree(self, tree, drive_id=None, level=0, verbose=True, indent = "|---"):
        cwd = os.getcwd()  # restore cwd after recursion is finished
        parent_folder = tree['title']
        os.chdir(parent_folder) 
            
        for f in tree['children']:
            if f['isfolder']:
                f['id'] = self.create_folder(f['title'], drive_id)
                if verbose: print(f"{indent*level}> {parent_folder}: FOLDER {f['title']} \t\t {f['id']}")
                self.upload_folder_tree(f, f['id'], level+1, verbose, indent)
            else:
                f['id'] = self.create_encrypted_file(f['title'], drive_id)
                if verbose: print(f"{indent*level}> {parent_folder}: FILE {f['title']} \t\t {f['id']}")
        
        os.chdir(cwd)
        return tree
    
    def download_folder_tree(self, fsave, tree, ignore=[], level=0, verbose=True, indent = "|---"):
        cwd = os.getcwd()  # restore cwd after recursion is finished
        parent_folder = tree['title']
        os.chdir(fsave) 
        
        for f in tree['children']:
            if f['isfolder']:
                mkdir(f['title'])
                if verbose: print(f"{indent*level}> {parent_folder}: FOLDER {f['title']} \t\t {f['id']}")
                self.download_folder_tree(f['title'], f, verbose=verbose, indent=indent, level=level+1)
            else:
                decript_stream_and_save(f['id'], self.bufferSize, self.password)
                if verbose: print(f"{indent*level}> {parent_folder}: FILE {f['title']} \t\t {f['id']}")
        
        os.chdir(cwd)
    
    def scan_drive_tree(self, path_id):
        dlist = driveListFolder(path_id)
        return driveList2localDict(dlist, path_id, "root", verbose=False)
        
    def scan_local_folder_tree(self, path):
        return localListFolder(path)

os.chdir("D:\\Code_Base\\pyDrive")
asd = DriveHandler()
dfiles = asd.scan_drive_tree(asd.root_id)
# pprint(dfiles)


asd.download_folder_tree('download', dfiles)
# files = localListFolder('.')
# pprint(files)
# dfiles = asd.upload_folder_tree(files)
# pprint(dfiles)

> root: FOLDER pyAesCrypt 		 15yXQqX35rt8RG9z2ZjyAqaSBFv3g0OGu
> pyAesCrypt: FILE __init__.py 		 1VzrPjr7F8naNOsRbGVbXfs8GlhB9Cpra
> pyAesCrypt: FILE test_crypto.py 		 10b7s0qP3iBoWoz4ljdsawzPDiHuaDIC9
> pyAesCrypt: FILE crypto.py 		 1UXiuvpIZTIbMQd1vbuxhqyBl2cplG-Xg
> root: FOLDER mypydrive 		 1AahvtKsON23QCpWy9V3sFfu0iogoOvdL
> mypydrive: FILE __init__.py 		 1FtE4ciUO6P3z7v4g82mh11QtXug0MiYm
> mypydrive: FOLDER test 		 18_6zahp24csAkQbO-dFwJ2sy93Kvals9
> test: FILE __init__.py 		 1U3vGOrlntnzrCQQDAoy8VVCJk-muWCX-
> test: FILE test_util.py 		 1EUI0UPqmP9m74kWzrIS8QiNGNL8HDk35
> test: FILE test_oauth.py 		 1JbNZXsFjOda-jl5_FJgCg1lizrgGyF32
> test: FILE test_filelist.py 		 1Ax7XUelLo5aPpqY59D8gHSVQbxZ3UGE0
> test: FILE test_file.py 		 11rLIvOokImFq5oATJjEFgKJE4lPUh_Hr
> test: FILE test_drive.py 		 1Wy3eIvt6Y1zrrr-ldIyQcP8_5yYgtf1S
> test: FILE test_apiattr.py 		 1S6VVowEZOLec6dHkrl7BufemuhIMIEm3
> test: FOLDER settings 		 1sjOdzmXHjKanp4jrXmfZtpJG5-nzFO-B
> settings: FILE test6.yaml 		 

In [ ]:
print(parent_id)
file2 = drive.CreateFile({'id': parent_id})
print('title: %s, mimeType: %s' % (file2['title'], file2['mimeType']))

In [ ]:

# driveListFolder('1YJkqN-qtRHXnL4GvT3icGyl05_dcYTE6')
os.chdir("D:\\Code_Base\\pyDrive")
asdd = localListFolder('.', verbose=True)

In [4]:
# l = os.listdir("./.ipynb_checkpoints")
# os.chdir("..")
# print([d for d in l if os.path.isdir(d)])
# print([d for d in l if not os.path.isdir(d)])
os.getcwd() 
dir(GoogleDrive)

['CreateFile',
 'GetAbout',
 'GetContentBinary',
 'ListFile',
 'SetContentBinary',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

# Tree of 3

In [1]:
import os
import re

## Tree

In [22]:
class Tree:
    
    def __init__(self, path, parent=None, level=None, type='folder'):
        self.path = path
        self.parent = parent
        self.type = type
        self.folders = {}
        self.files = {}
        self.level = level
    
    @property
    def folder_list(self):
        return list(self.folders.keys())
    
    @property
    def file_list(self):
        return list(self.files.keys())
    
    @property
    def child_list(self):
        return self.folder_list + self.file_list 
    
    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        s = "~"*20 + "\n"
        if self.parent is not None: 
            s += f"(parent.path, {self.parent.__getitem__('path')})\n"
        else:
            s += f"(parent.path, None)\n"
        
        for k, v in self.__dict__.items():
            if k not in ['parent', "folders", "files"]:
                if v is not None: s += f"{k, v}\n"
        
        s += f"(child_list, {self.child_list})"
        s += "\n" +"~"*20 + "\n"
        return s
    
    def __sub__(self, other):
        file_diff = set(self.file_list).difference(set(other.file_list))
        folder_diff = set(self.file_list).difference(set(other.file_list)) 
        
    
    def __setitem__(self, k, v):
        self.__setattr__(k, v)
        
    def __getitem__(self, k):
        try:
            return self.__getattribute__(k)
        except:
            if k in self.folder_list:
                return self.folders[k]
            elif k in self.file_list:
                return self.files[k]
            else:
                return None

In [3]:
def localListFolder(path, parent=None, level=0, verbose=False, ignore=None):
    indent = ''
    file_list = [asd.name for asd in os.scandir(path) if not asd.is_dir() and not re.match(r'[\d~_.]', asd.name)]
    folder_list = [asd.name for asd in os.scandir(path) if asd.is_dir()]
    
    tree = Tree(path, parent, level=level)
    for f in folder_list:
        if verbose: print(f"{indent*level}>{parent}:\t|{f}|")
        tree['folders'][f] = localListFolder(f"{path}\\{f}", tree, level=level+1, verbose=verbose)
    else:
        tree['files'] = {f: Tree(f, parent=tree, level=level, type="file") for f in file_list}
        
    return tree



    

In [7]:
root = r".\mypydrive_1"
struct = localListFolder(root, verbose=False)
print(struct)

~~~~~~~~~~~~~~~~~~~~
(parent.path, None)
('path', '.\\mypydrive_1')
('type', 'folder')
('level', 0)
(child_list, ['.ipynb_checkpoints', 'test', 'apiattr.py', 'auth.py', 'drive.py', 'files.py', 'settings.py'])
~~~~~~~~~~~~~~~~~~~~



In [5]:
def navigate(current):
    while True:
        try:
            print(current)
            print(current.child_list)
            sel = input('Navigate: ')
            if sel in current['child_list']:
                current = current[sel]
            elif sel == "back":
                current = current["parent"]
        except KeyboardInterrupt:
            break
navigate(struct) 

~~~~~~~~~~~~~~~~~~~~
(parent.path, None)
('path', '.\\mypydrive')
('type', 'folder')
('level', 0)
(child_list, ['.ipynb_checkpoints', 'test', '__pycache__', 'apiattr.py', 'auth.py', 'drive.py', 'files.py', 'settings.py'])
~~~~~~~~~~~~~~~~~~~~

['.ipynb_checkpoints', 'test', '__pycache__', 'apiattr.py', 'auth.py', 'drive.py', 'files.py', 'settings.py']


Navigate:  1


~~~~~~~~~~~~~~~~~~~~
(parent.path, None)
('path', '.\\mypydrive')
('type', 'folder')
('level', 0)
(child_list, ['.ipynb_checkpoints', 'test', '__pycache__', 'apiattr.py', 'auth.py', 'drive.py', 'files.py', 'settings.py'])
~~~~~~~~~~~~~~~~~~~~

['.ipynb_checkpoints', 'test', '__pycache__', 'apiattr.py', 'auth.py', 'drive.py', 'files.py', 'settings.py']


Navigate:  test


~~~~~~~~~~~~~~~~~~~~
(parent.path, .\mypydrive)
('path', '.\\mypydrive\\test')
('type', 'folder')
('level', 1)
(child_list, ['settings', 'README.rst', 'test_apiattr.py', 'test_drive.py', 'test_file.py', 'test_filelist.py', 'test_oauth.py', 'test_util.py'])
~~~~~~~~~~~~~~~~~~~~

['settings', 'README.rst', 'test_apiattr.py', 'test_drive.py', 'test_file.py', 'test_filelist.py', 'test_oauth.py', 'test_util.py']


Navigate:  back


~~~~~~~~~~~~~~~~~~~~
(parent.path, None)
('path', '.\\mypydrive')
('type', 'folder')
('level', 0)
(child_list, ['.ipynb_checkpoints', 'test', '__pycache__', 'apiattr.py', 'auth.py', 'drive.py', 'files.py', 'settings.py'])
~~~~~~~~~~~~~~~~~~~~

['.ipynb_checkpoints', 'test', '__pycache__', 'apiattr.py', 'auth.py', 'drive.py', 'files.py', 'settings.py']


Navigate:  back


None


AttributeError: 'NoneType' object has no attribute 'child_list'

## Diff tree

In [9]:
root_1 = r".\mypydrive_1"
struct_1 = localListFolder(root_1, verbose=False)
root_2 = r".\mypydrive_2"
struct_2 = localListFolder(root_2, verbose=False)

In [21]:
difference = set(struct_1.file_list).difference(set(struct_2.file_list))
difference
set(struct_1.folder_list).symmetric_difference(set(struct_2.folder_list))

{'.ipynb_checkpoints', '__pycache__'}

In [ ]:
def diff(tree_1, tree_2):
    